In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (8,176 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120874 files and dire

In [ ]:
import matplotlib.pyplot as plt # 맷플롯립 import 하기
plt.rcParams['font.family'] = 'NanumBarunGothic' # 나눔바른고딕 적용하기

In [ ]:
!pip install haversine

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
from haversine import haversine
from os.path import join
import math
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import requests
import json
import re

In [ ]:
uni = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/통계학과경진대회/데이터/대학교_위경도(폐교제외).csv', encoding='utf-8',)
subway_station = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/통계학과경진대회/데이터/지하철 역 (서울 기준, 중복 제거 안함).csv', encoding='utf-8')
apart = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/통계학과경진대회/데이터/서울아파트_위경도포함.csv',encoding='cp949')
bic = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/통계학과경진대회/데이터/서울시공공자전거대여소위도경도.csv',encoding='utf-8')
old = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/통계학과경진대회/데이터/교내통계경진대회데이터셋-노인복지시설.csv',encoding='utf-8')

In [ ]:
subway_station.head()

,호선명,지하철역,위도,경도,상세주소
0,1호선,가산디지털단지,37.481072,126.882343,서울 금천구 가산동 371-28
1,1호선,개봉,37.494594,126.858680,서울 구로구 개봉동 415
2,1호선,광운대,37.623632,127.061835,서울 노원구 월계동 85
3,1호선,구로,37.503039,126.881966,서울 구로구 구로동 589-14
4,1호선,구일,37.496756,126.870793,서울 구로구 구로동 636-45


In [ ]:
uni.head()

,키값,학교종류,학교명,본분교,학교상태,설립,주소,행정시,행정구,행정동,위도,경도
0,BE_LiST23-0034,일반대학,서울시립대학교,본교,기존,공립,서울 동대문구 서울시립대로 163 (전농동 90번지),서울특별시,동대문구,휘경2동,37.582578,127.059988
1,BE_LiST23-0035,전문대학(3년제),서울여자간호대학교,본교,기존,사립,서울 서대문구 홍제3동 서울여자간호대학,서울특별시,서대문구,홍제2동,37.597030,126.956477
2,BE_LiST23-0036,일반대학,서울여자대학교,본교,기존,사립,서울특별시 노원구 화랑로 621 서울여자대학교,서울특별시,노원구,공릉2동,37.627382,127.091621
3,BE_LiST23-0037,전문대학(2년제),서일대학교,본교,학교명 변경,사립,서울 중랑구 서일대학길 22(면목동 49-3) 서일대학교,서울특별시,중랑구,면목제3.8동,37.568215,126.964002
4,BE_LiST23-0038,일반대학,성공회대학교,본교,기존,사립,서울 구로구 항동 성공회대학교,서울특별시,구로구,오류2동,37.488462,126.824748


In [ ]:
apart.head()

,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시도)k-apt주소split,주소(시군구),주소(읍면동),나머지주소,주소(도로명),...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,경도,위도,단지신청일
0,4961,B11680001,일원대우,NaN,NaN,서울,강남구,일원동,690-1,개포로110길,...,직영,1886.00,130.0,임의,2017/03/31 16:59,Y,N,126.964002,37.568215,2015/02/10 11:11
1,5143,B11680020,삼성동롯데캐슬킹덤,NaN,NaN,서울,강남구,삼성동,롯데캐슬킹덤아파트,선릉로112길,...,위탁+직영,9108.30,276.0,임의,2016/06/16 9:51,Y,N,127.048185,37.512859,2015/02/27 9:51
2,5164,B11680041,도곡3차아이파크,NaN,NaN,서울,강남구,도곡동,도곡3차아이파크관리사무소,도곡로14길,...,위탁+직영,2054.94,199.0,임의,2017/10/24 16:43,Y,N,126.964002,37.568215,2015/02/27 9:51
3,5183,B11680060,도곡2차 I-Park,NaN,NaN,서울,강남구,도곡동,관리사무소,도곡로14길,...,직영,1101.70,120.0,임의,2019/02/26 11:38,Y,N,127.037311,37.489186,2015/02/27 9:51
4,7001,B20160929,청담4차e편한세상,NaN,NaN,서울특별시,강남구,청담동,33 청담4차e편한세상 아파트,학동로73길,...,위탁,1326.95,158.0,임의,2016/09/29 10:37,Y,N,127.047840,37.519435,2016/09/29 10:18


In [ ]:
old.head()

,이름,주소,상세주소,경도,위도
0,강남구립 강남시니어플라자,역삼1동,서울 강남구 봉은사로 332,127.043218,37.509787
1,강남구립 역삼노인복지센터,역삼2동,서울특별시 강남구도곡로27길 27 (역삼동),127.041810,37.494558
2,강남구립 압구정노인복지센터,압구정동,서울특별시 강남구도산대로53길 5 (신사동),127.038159,37.523551
3,강남구립 강남노인종합복지관,삼성1동,서울 강남구 삼성로 628,127.052143,37.515911
4,강남구립 논현노인종합복지관,논현1동,서울 강남구 강남대로128길 59,127.026089,37.512065


In [ ]:
subway_station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   호선명     324 non-null    object 
 1   지하철역    324 non-null    object 
 2   위도      324 non-null    float64
 3   경도      324 non-null    float64
 4   상세주소    324 non-null    object 
dtypes: float64(2), object(3)
memory usage: 12.8+ KB


In [ ]:
arr = []
for i in tqdm(range(len(subway_station))):
    apt_cnt = 0
    uni_cnt = 0
    bic_cnt = 0
    #지하철 위도 경도 추출
    subway = (subway_station.iloc[i][2], subway_station.iloc[i][3])

    for j in range(0, len(uni)):
        # 대학교 위도 경도 추출
        uni_xy = (uni.iloc[j][10], uni.iloc[j][11])

        # 거리 계산
        uni_dist = haversine(subway, uni_xy, unit='m')

        #거리가 1.5킬로미터면 count
        if int(uni_dist) <= 1000:
            uni_cnt += 1
    arr.append([ subway_station.iloc[i][1], subway[0], subway[1], uni_cnt])

savedf = pd.DataFrame(
    arr, columns=['지하철역', 'latitude', 'longitude', '대학교 수'])

100%|██████████| 324/324 [00:03<00:00, 96.01it/s]


In [ ]:
arr = []
for i in tqdm(range(len(subway_station))):
    apt_cnt = 0
    uni_cnt = 0
    bic_cnt = 0
    #지하철 위도 경도 추출
    subway = (subway_station.iloc[i][2], subway_station.iloc[i][3])

    for j in range(0, len(apart)):

      #아파트 위도 경도 추출
        apart_xy = (apart.iloc[j][-2], apart.iloc[j][-3])

        # 거리 계산
        apart_dist = haversine(subway, apart_xy, unit='m')

        #거리가 5백미터면 count
        if int(apart_dist) <= 500:
            apt_cnt += 1

    for j in range(0, len(uni)):
        # 대학교 위도 경도 추출
        uni_xy = (uni.iloc[j][10], uni.iloc[j][11])

        # 거리 계산
        uni_dist = haversine(subway, uni_xy, unit='m')

        #거리가 1.5킬로미터면 count
        if int(uni_dist) <= 1500:
            uni_cnt += 1

    for j in range(0, len(bic)):
        bic_xy = (bic.iloc[j][3], bic.iloc[j][4])
        # 거리 계산
        bic_dist = haversine(subway, bic_xy, unit='m')
         #거리가 500미터면 count
        if int(bic_dist) <= 500:
            bic_cnt += 1
    arr.append([ subway_station.iloc[i][1], subway[0], subway[1],apt_cnt, uni_cnt,bic_cnt])

savedf = pd.DataFrame(
    arr, columns=['지하철역', 'latitude', 'longitude','아파트 수', '대학교 수','공공 자전거 시설 수'])

100%|██████████| 324/324 [06:05<00:00,  1.13s/it]


In [ ]:
savedf.head()

,지하철역,latitude,longitude,아파트 수,대학교 수,공공 자전거 시설 수
0,가산디지털단지,37.481072,126.882343,0,0,9
1,개봉,37.494594,126.858680,11,2,7
2,광운대,37.623632,127.061835,8,3,3
3,구로,37.503039,126.881966,5,2,5
4,구일,37.496756,126.870793,5,1,8


In [ ]:
savedf.to_csv('/content/drive/MyDrive/Colab Notebooks/통계학과경진대회/데이터/count.csv')

In [ ]:
savedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   지하철역         324 non-null    object 
 1   latitude     324 non-null    float64
 2   longitude    324 non-null    float64
 3   아파트 수        324 non-null    int64  
 4   대학교 수        324 non-null    int64  
 5   공공 자전거 시설 수  324 non-null    int64  
 6   노인 복지시설 수    324 non-null    int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 17.8+ KB


In [ ]:
savedf[savedf['노인 복지시설 수']==0]

,지하철역,latitude,longitude,아파트 수,대학교 수,공공 자전거 시설 수,노인 복지시설 수
0,가산디지털단지,37.481072,126.882343,0,0,9,0
1,개봉,37.494594,126.858680,11,2,7,0
2,광운대,37.623632,127.061835,8,3,3,0
3,구로,37.503039,126.881966,5,2,5,0
4,구일,37.496756,126.870793,5,1,8,0
...,...,...,...,...,...,...,...
319,종합운동장,37.510997,127.073642,2,0,4,0
320,중앙보훈병원,37.528879,127.148536,5,0,3,0
321,증미,37.557402,126.861939,17,0,5,0
322,한성백제,37.516421,127.116248,0,1,5,0
